Задание 1

In [1]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt

--2019-11-22 11:58:30--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123167 (120K) [text/plain]
Saving to: ‘sents_with_mistakes.txt’

sents_with_mistakes 100%[===================>] 120.28K  --.-KB/s    in 0.04s   

2019-11-22 11:58:30 (3.08 MB/s) - ‘sents_with_mistakes.txt’ saved [123167/123167]

--2019-11-22 11:58:32--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awa

In [0]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)

from sklearn.metrics import classification_report, accuracy_score

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [0]:

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [31]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [32]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_500.txt

--2019-11-22 11:59:55--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_500.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1181403 (1.1M) [text/plain]
Saving to: ‘corpus_500.txt.1’

corpus_500.txt.1    100%[===================>]   1.13M  --.-KB/s    in 0.09s   

2019-11-22 11:59:55 (12.9 MB/s) - ‘corpus_500.txt.1’ saved [1181403/1181403]



In [0]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [34]:
corpus = []
for text in open('corpus_500.txt').read().split():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

print(corpus[:10])

[['text'], ['вице-премьер'], ['по'], ['социальным'], ['вопросам'], ['татьяна'], ['голикова'], ['рассказала'], ['в'], ['каких']]


In [0]:
def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

    
    

In [0]:
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [37]:
WORDS.most_common(10)


[('в', 4263),
 ('и', 1999),
 ('на', 1757),
 ('что', 1236),
 ('с', 990),
 ('по', 850),
 ('не', 739),
 ('он', 425),
 ('из', 406),
 ('этом', 405)]

In [0]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def Pos(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [0]:
dict2 = {}
for key in WORDS:
  splits     =  [(key[:i], key[i:])    for i in range(len(key) + 1)]
  deletes1    = [L + R[1:]               for L, R in splits if R]
  splits2     = [(key[:i], key[i:])    for i in range(len(deletes1))]
  deletes2    = [L + R[2:]               for L, R in splits2 if R]
  deletes = deletes1 + deletes2
  for element in deletes: 
    dict2[element] = key
 



In [0]:
def compare(word):
  cand = []
  if word in WORDS:
    return word
  elif word in dict2:
    cand.append(dict2[word])
  else:
    splits     =  [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes1    = [L + R[1:]               for L, R in splits if R]
    splits2     = [(word[:i], word[i:])    for i in range(len(deletes1))]
    deletes2    = [L + R[2:]               for L, R in splits2 if R]
    deletes = deletes1 + deletes2
    for delts in deletes:
      if delts in WORDS:
        cand.append(delts)
      elif delts in dict2:
        cand.append(dict2[delts])
      else:
        continue

  if cand:
    return max(cand, key=Pos)
  else:
    return 'Нельзя исправить'

In [255]:
%%time
compare('онор')

CPU times: user 100 µs, sys: 14 µs, total: 114 µs
Wall time: 120 µs


'он'

In [253]:
%%time
print(compare('мш,ина'))
print(compare('ручноой'))
print(compare('речнйо'))
print(compare('седня'))
print(compare('оочень'))


машина
ручной
речного
сегодня
очень
CPU times: user 713 µs, sys: 0 ns, total: 713 µs
Wall time: 669 µs


In [256]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], compare(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [257]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.5247752247752248
0.3461243284727552
0.44848972091420697


Качество стало хуже. Достаточно высокий показатель неправильно исправленных правильных слов. Может быть, проблема кроется в том, что в выбранном корпусе много коротких слов, которые сокращаются до последовательностьй в один-два знака, которые затем выискиваются в словаре ошибок. При этом выдаётся самое частотное слово. Выше есть пример с 'онор', которое исправляется на 'он', вместо 'оно', просто потому, что слово 'он' более частотное. С этим, наверное, можно бороться, назначив приоритет слову с одним удалением. 

Задание 2

In [45]:
corpus2 = []
for text in open('corpus_500.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus2 += norm_sents

print(corpus2[:10])

[['text'], ['вице-премьер', 'по', 'социальным', 'вопросам', 'татьяна', 'голикова', 'рассказала', 'в', 'каких', 'регионах', 'россии', 'зафиксирована', 'наиболее', 'высокая', 'смертность', 'от', 'рака', 'сообщает', 'риа', 'новости'], ['по', 'словам', 'голиковой', 'чаще', 'всего', 'онкологические', 'заболевания', 'становились', 'причиной', 'смерти', 'в', 'псковской', 'тверской', 'тульской', 'и', 'орловской', 'областях', 'а', 'также', 'в', 'севастополе'], ['вице-премьер', 'напомнила', 'что', 'главные', 'факторы', 'смертности', 'в', 'россии', 'рак', 'и', 'болезни', 'системы', 'кровообращения'], ['в', 'начале', 'года', 'стало', 'известно', 'что', 'смертность', 'от', 'онкологических', 'заболеваний', 'среди', 'россиян', 'снизилась', 'впервые', 'за', 'три', 'года'], ['по', 'данным', 'росстата', 'в', '2017', 'году', 'от', 'рака', 'умерли', '289', 'тысяч', 'человек'], ['это', 'на', '3,5', 'процента', 'меньше', 'чем', 'годом', 'ранее'], ['австрийские', 'правоохранительные', 'органы', 'не', 'предст

In [0]:
def ngrammer2(tokens, n=2):
    ngrams2 = []
    for i in range(0,len(tokens)-n+1):
        ngrams2.append(' '.join(tokens[i:i+n]))
    return ngrams2
def ngrammer3(tokens, n=3):
    ngrams3 = []
    for i in range(0,len(tokens)-n+1):
        ngrams3.append(' '.join(tokens[i:i+n]))
    return ngrams3

In [0]:
corpus_sents = [['<start>', '<start>'] + sent + ['<end>'] for sent in corpus2]

In [0]:
unigrams = Counter()
bigrams= Counter()
trigrams = Counter()

for sentence in corpus_sents:
    unigrams.update(sentence)
    bigrams.update(ngrammer2(sentence))
    trigrams.update(ngrammer3(sentence))

In [49]:
trigrams.most_common(10)


[('<start> <start> в', 710),
 ('<start> <start> по', 323),
 ('<start> <start> об', 296),
 ('<start> об этом', 290),
 ('<start> <start> он', 132),
 ('<start> <start> на', 132),
 ('об этом сообщает', 125),
 ('<start> <start> ранее', 102),
 ('<start> по его', 75),
 ('<start> по словам', 73)]

In [0]:
dict2_unigrams = {}
for word in unigrams:
  splits     =  [(word[:i], word[i:])    for i in range(len(word) + 1)]
  deletes1    = [L + R[1:]               for L, R in splits if R]
  splits2     = [(word[:i], word[i:])    for i in range(len(deletes1))]
  deletes2    = [L + R[2:]               for L, R in splits2 if R]
  deletes = deletes1 + deletes2
  for element in deletes: 
    dict2_unigrams[element] = word

In [0]:
dict2_bigrams = {}
for word in bigrams:
  splits     =  [(word[:i], word[i:])    for i in range(len(word) + 1)]
  deletes1    = [L + R[1:]               for L, R in splits if R]
  splits2     = [(word[:i], word[i:])    for i in range(len(deletes1))]
  deletes2    = [L + R[2:]               for L, R in splits2 if R]
  deletes = deletes1 + deletes2
  for element in deletes: 
    dict2_bigrams[element] = word

In [0]:
dict2_trigrams = {}
for word in trigrams:
  splits     =  [(word[:i], word[i:])    for i in range(len(word) + 1)]
  deletes1    = [L + R[1:]               for L, R in splits if R]
  splits2     = [(word[:i], word[i:])    for i in range(len(deletes1))]
  deletes2    = [L + R[2:]               for L, R in splits2 if R]
  deletes = deletes1 + deletes2
  for element in deletes: 
    dict2_trigrams[element] = word


In [0]:
def poser(delts):
  delt = str(delts.split()[:-1])
  if delt in WORDS:
    return Pos(delt)
  elif delt in dict2:
    delts_pos = unigrams[delt]/len(unigrams)
    bigram = " ".join(delt)
    if WORDS[delts.split()[0]]:
      bigram_pos = bigrams[bigram]/unigrams[delts.split()[0]]
      trigram = bigram + " " + dict2[delt]
    if bigrams[bigram]:
      trigram_pos = trigrams[trigram]/bigrams[bigram]
      context_pos = delts_pos*(1 + trigram_pos/bigram_pos)
      return context_pos
    else:
      context_pos = delts_pos
    return context_pos
  else:
    return 0
  

In [0]:
def correct_grams(word):
  cand = []
  if word in WORDS:
    return word
  elif word in dict2:
    cand.append(dict2[word])
  elif word in dict2_bigrams:
    cand.append(dict2_bigrams[word])
  elif word in dict2_trigrams:
    cand.append(dict2_trigrams[word])  
  else:
    splits     =  [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes1    = [L + R[1:]               for L, R in splits if R]
    splits2     = [(word[:i], word[i:])    for i in range(len(deletes1))]
    deletes2    = [L + R[2:]               for L, R in splits2 if R]
    deletes3 = deletes1 + deletes2

    word1 = str(word.split()[-1])
    splits3     =  [(word1[:i], word1[i:])    for i in range(len(word1) + 1)]
    deletes4    = [L + R[1:]               for L, R in splits3 if R]
    splits4     = [(word1[:i], word1[i:])    for i in range(len(deletes4))]
    deletes5    = [L + R[2:]               for L, R in splits4 if R]
    deletes6 = deletes3 + deletes4 + deletes5

    for delts in deletes6:
      if delts in WORDS:
        cand.append(delts)
      elif delts in dict2:
        cand.append(dict2[delts])
      elif delts in dict2_bigrams:
        cand.append(dict2_bigrams[delts])
      elif delts in dict2_trigrams:
        cand.append(dict2_trigrams[delts])


  if cand:
    return max(cand, key=poser)
  else:
    return 'Нельзя исправить'

In [252]:
print(compare('какт'))
print(correct_grams('какт'))
print(correct_grams('ывлоаыаоыола'))

как
как от
Нельзя исправить


In [248]:
%%time
print(correct_grams('мшина'))
print(correct_grams('яптницу 14'))
print(correct_grams('сосылкой н'))
print(correct_grams('об этам сообщает'))
print(correct_grams('общает'))
print(correct_grams('как ообщает'))


машина
пятницу 14
ссылкой на
об этом сообщает
общается
как сообщает
CPU times: user 2.58 ms, sys: 79 µs, total: 2.66 ms
Wall time: 7.23 ms


In [259]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], correct_grams(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [260]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.509090909090909
0.3215656178050652
0.4628459859882853


С нграмной моделью упало количество правильно исправленных слов и возросло количество неправильно исправленных, что не есть хорошо. Можно сделать вывод, что качество упало. Проблема может быть в том, что алгоритм с большей вероятностью выбирает нграмм, в составе которого есть нужное слово (потому что урезанные на несколько знаков нграммы могут добавлятся сразу на нескольких этапах). К тому же, в алгоритме симспелл все операции равнозначны, так как завязаны на одних только удалениях. Возможно поэтому и происходят всякие казусы типа испаравления 'какт' на 'как от', вместо 'как-то' или 'как'. 
То есть, основная причина падения качества - добавление словарей с изуродованными нграммами, что хорошо, для исправления некоторых ошибок внутри словосочетаний, но, в целом, негативно влияет на общее качество.   